In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import seaborn as sns
np.random.seed(42)

In [2]:
conts = pd.read_csv('../data/documentContents.csv')
header = ['seed'] + ['rec'+str(i) for i in range(14)]
recs = pd.read_csv('../data/recommendationPairs.csv',names = header,index_col=0)
train_ids = pd.read_csv('../data/dataTrainIDs.csv')

full= pd.read_csv('../data/out.csv')

In [3]:
conts = conts.rename(columns={'Abstract/Review/Summarry':'text'})
conts = conts[~conts.text.isna()]
conts['text'] = conts.text.apply(lambda x: x[2:-2])
conts['text_len'] = conts.text.apply(lambda x: len(x))
conts = conts[conts.text_len > 150]
conts['zbMATH_ID']=conts['zbMATH_ID'].astype(int)
conts=conts.set_index('zbMATH_ID')

In [4]:
full = full.rename(columns={'de':'id'})[['id','text']]
full['text_len'] = full.text.apply(lambda x: len(x))
full = full[full.text_len > 150]
full['id']=full['id'].astype(int)
full=full.set_index('id')

In [5]:
train_ids = train_ids['seeID'].unique()
test_ids = np.setdiff1d(np.array(recs.index),train_ids)
pool_ids = np.array(full.index)
all_labeled_ids = np.array(recs.index)

In [6]:
train_recs = recs.loc[train_ids]
test_recs = recs.loc[test_ids]

In [7]:
def create_pairs(df,recs_ids,full_ids):
    rowlist=[]
    for seed,row in tqdm(df.iterrows()):
        rec_list = row[~row.isna()].astype(int).to_list()
        for rec_id in rec_list:
            # create positive
            rowlist.append({'seed':seed,'rec':rec_id,'label':1})
            # sample negative
            random_negative = np.random.choice(full_ids)
            while random_negative in recs_ids:
                random_negative = np.random.choice(full_ids)
            rowlist.append({'seed':seed,'rec':random_negative,'label':0})
    return pd.DataFrame(rowlist)
    

In [8]:
train_dataset = create_pairs(train_recs,all_labeled_ids,full_ids=pool_ids)
test_dataset = create_pairs(test_recs,all_labeled_ids,full_ids=pool_ids)

20it [00:00, 1390.41it/s]


60it [00:00, 1902.83it/s]


In [9]:
train_dataset

,seed,rec,label
0,1566951,4181495,1
1,1566951,5924715,0
2,1566951,930151,1
3,1566951,2507805,0
4,1566951,5083606,1
...,...,...,...
223,1269765,1233903,0
224,1269765,952018,1
225,1269765,2541827,0
226,1269765,1334957,1


In [10]:
def get_contents(df,lookup_positive,lookup_negative):
    rowlist=[]
    for idx,row in df.iterrows():

        if row['seed'] not in lookup_positive.index:
            continue
        anchor = lookup_positive.loc[row['seed']].text
        # positive recommendations
        if row['label']==1:
            
            # skip the ones that are not present in contents
            if row['rec'] not in lookup_positive.index:
                continue
            
            rec = lookup_positive.loc[row['rec']].text
            
        # negative ones. Sampling from whole zbmath corpus
        else:
            rec = lookup_negative.loc[row['rec']].text

        rowlist.append({'seed':row['seed'],'anchor':anchor,'rec':rec,'label':row['label']})
    return pd.DataFrame(rowlist).set_index('seed').sample(frac=1.,random_state=42)
        

In [11]:
final_train_dataset = get_contents(train_dataset,conts,full)
final_test_dataset = get_contents(test_dataset,conts,full)

In [12]:
final_train_dataset.iloc[45]

anchor    Summary: Let \\(G\\) be a finite cyclic subgro...
rec       zbMATH_ID
1341964    The article consists of t...
label                                                     1
Name: 1619869, dtype: object

In [13]:
final_train_dataset

,anchor,rec,label
seed,,,
1566951,"Let \\(k\\) be a field, and let \\(E\\) be a f...",Let \(M\) be the set of all finite complex-val...,0
1745734,The author considers the following two familie...,"Zweck der Arbeit ist die Untersuchung, in wie ...",0
1664684,Summary: We summarise the brane solutions (ins...,"Let \(G\) be a finite group and \((K,R,k)\) a ...",0
1275776,Voloch and Brownawell and Masser proved the ``...,"Let \\(k\\) be a field of characteristic 0, an...",1
1058016,This is a very well written survey of the stat...,In this paper the retina R is modelled as a tw...,0
...,...,...,...
1341964,zbMATH_ID 1341964 The article consists of t...,"Es wird der Einfluß untersucht, den das Schief...",0
1363213,zbMATH_ID 1363213 Let \\(X\\) be a smooth c...,Let \\(X\\) be a smooth algebraic surface over...,1
1341964,zbMATH_ID 1341964 The article consists of t...,\textit{Hardy} hat die Integrale \n\[\n\begin{...,0


In [14]:
final_train_dataset.to_csv('../data/final_train_dataset.csv')
final_test_dataset.to_csv('../data/final_train_dataset.csv')